In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip /content/drive/MyDrive/Graduation_Project/CheXpert-v1.0-small.zip > /dev/null

In [3]:
!pip install libauc > /dev/null

In [4]:
pip install timm

     |████████████████████████████████| 431 kB 4.2 MB/s 


In [5]:
!pip install kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!kaggle datasets download -d nih-chest-xrays/sample

100% 4.19G/4.20G [01:36<00:00, 80.0MB/s]
100% 4.20G/4.20G [01:36<00:00, 46.8MB/s]


In [7]:
!unzip /content/sample.zip > /dev/null

In [8]:
import cv2
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

import torch
from torch import nn
from torchsummary import summary

import timm
from chexpert_v4 import CheXpert
from libauc.optimizers import PESG
from libauc.losses import AUCM_MultiLabel

from conv1dnet_v2 import WholeConv1dArch, Conv1dArch

from sklearn.metrics import roc_auc_score

In [9]:
save_weights_path = '/content/drive/MyDrive/Graduation_Project/conv1d_pretrained_2.pth'
load_weights_path = '/content/drive/MyDrive/Graduation_Project/conv1d_pretrained.pth'

In [10]:
avail_pretrained_models = timm.list_models(pretrained=True)
avail_pretrained_models

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_base',
 'convnext_base_384_in22ft1k',
 'convnext_base_in22ft1k',
 'convnext_base_in22k',
 'convnext_large',
 'convnext_large_384_in22ft1k',
 'convnext_large_in22ft1k',
 'convnext_large_in22k',
 'convnext_small',
 'convnext_tiny',
 'convnext_xlarge_384_in22ft1k',
 'convnext_xlarge_in22ft1k',
 'convnext_xlarge_in22k',
 'crossvit_9_240',
 'crossv

In [11]:
model_1 = timm.create_model('rexnet_100', num_classes=5, pretrained=False)
model_1.load_state_dict(torch.load(r'/content/drive/MyDrive/Graduation_Project/new_pretrained_2 (rexnet _v4_).pth'))
model_1 = nn.Sequential(*list(model_1.children())[:-1], nn.AdaptiveAvgPool2d(output_size=1), nn.Flatten())
model_1 = model_1.cuda()

model_2 = timm.create_model('tf_mobilenetv3_large_100', num_classes=5, pretrained=False)
model_2.load_state_dict(torch.load(r'/content/drive/MyDrive/Graduation_Project/new_pretrained_2 (mobilenet _v4_).pth'))
model_2 = nn.Sequential(*list(model_2.children())[:-1])
model_2 = model_2.cuda()

conv1d = Conv1dArch(5)
conv1d = conv1d.cuda()

In [12]:
for param in model_1.parameters():
    param.requires_grad = False

for param in model_2.parameters():
    param.requires_grad = False

In [13]:
model = WholeConv1dArch(model_1, model_2, conv1d)
model = model.cuda()

In [14]:
summary(model, (3, 224, 224), batch_size=1, device='cuda')

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1          [1, 32, 112, 112]             864
              SiLU-2          [1, 32, 112, 112]               0
    BatchNormAct2d-3          [1, 32, 112, 112]              64
         ConvBnAct-4          [1, 32, 112, 112]               0
            Conv2d-5          [1, 32, 112, 112]             288
          Identity-6          [1, 32, 112, 112]               0
    BatchNormAct2d-7          [1, 32, 112, 112]              64
         ConvBnAct-8          [1, 32, 112, 112]               0
             ReLU6-9          [1, 32, 112, 112]               0
           Conv2d-10          [1, 16, 112, 112]             512
         Identity-11          [1, 16, 112, 112]               0
   BatchNormAct2d-12          [1, 16, 112, 112]              32
        ConvBnAct-13          [1, 16, 112, 112]               0
 LinearBottleneck-14          [1, 16, 1

In [15]:
# dataloader
root = '/content/'

trainGen = CheXpert(image_root_path=root, 
                     csv_path='clean_data_1.csv', 
                     image_size=224)

valGen =  CheXpert(image_root_path=root, 
                    csv_path='valid_5.csv', 
                    image_size=224)

trainloader =  torch.utils.data.DataLoader(trainGen, batch_size=32, num_workers=2, shuffle=True)
testloader =  torch.utils.data.DataLoader(valGen, batch_size=32, num_workers=2, shuffle=False)

lr = 0.1 
gamma = 1000
imratio = trainGen.imratio_list 
weight_decay = 1e-4
margin = 1.0


# define loss & optimizer
Loss = AUCM_MultiLabel(imratio=imratio, num_classes=5)
optimizer = PESG(model, 
                 a=Loss.a, 
                 b=Loss.b, 
                 alpha=Loss.alpha, 
                 lr=lr, 
                 gamma=gamma, 
                 margin=margin, 
                 weight_decay=weight_decay, device='cuda')

#model.load_state_dict(torch.load(load_weights_path))

# training
#diff_threshold = 0.01
const_threshold = 5

val_auc_mean = 0
best_val_auc = 0
const_cnt = 0
for epoch in range(5):

    for idx, data in enumerate(trainloader):
      train_data, train_labels = data
      train_data, train_labels  = train_data.cuda(), train_labels.cuda()
      y_pred = model(train_data)
      y_pred = torch.sigmoid(y_pred)
      loss = Loss(y_pred, train_labels)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
        
      # validation  
      if idx % 400 == 0:
         model.eval()
         with torch.no_grad():    
              test_pred = []
              test_true = [] 
              for data in testloader:
                  test_data, test_labels = data
                  test_data = test_data.cuda()
                  y_pred = model(test_data)
                  test_pred.append(y_pred.cpu().detach().numpy())
                  test_true.append(test_labels.numpy())
            
              test_true = np.concatenate(test_true)
              test_pred = np.concatenate(test_pred)
              val_auc_mean =  roc_auc_score(test_true, test_pred) 
              model.train()

              if best_val_auc < val_auc_mean:
                  const_cnt = 0
                  best_val_auc = val_auc_mean
                  torch.save(model.state_dict(), save_weights_path)
              else:
                  const_cnt += 1            

              #val_diff = val_auc_mean-best_val_auc
              if (const_cnt >= const_threshold):
                  optimizer.update_regularizer(decay_factor=2)
                  model.load_state_dict(torch.load(save_weights_path))
                  const_cnt = 0 

              print ('Epoch=%s, BatchID=%s, Val_AUC=%.4f, Best_Val_AUC=%.4f'%(epoch, idx, val_auc_mean, best_val_auc))

Epoch=0, BatchID=0, Val_AUC=0.4177, Best_Val_AUC=0.4177
Epoch=0, BatchID=400, Val_AUC=0.8695, Best_Val_AUC=0.8695
Epoch=0, BatchID=800, Val_AUC=0.8813, Best_Val_AUC=0.8813
Epoch=0, BatchID=1200, Val_AUC=0.8463, Best_Val_AUC=0.8813
Epoch=0, BatchID=1600, Val_AUC=0.8645, Best_Val_AUC=0.8813
Epoch=0, BatchID=2000, Val_AUC=0.8549, Best_Val_AUC=0.8813
Epoch=0, BatchID=2400, Val_AUC=0.8802, Best_Val_AUC=0.8813
Reducing learning rate to 0.05000 @ T=2801!
Updating regularizer @ T=2801!
Epoch=0, BatchID=2800, Val_AUC=0.8445, Best_Val_AUC=0.8813
Epoch=0, BatchID=3200, Val_AUC=0.8742, Best_Val_AUC=0.8813
Epoch=0, BatchID=3600, Val_AUC=0.8552, Best_Val_AUC=0.8813
Epoch=0, BatchID=4000, Val_AUC=0.8684, Best_Val_AUC=0.8813
Epoch=0, BatchID=4400, Val_AUC=0.8698, Best_Val_AUC=0.8813
Reducing learning rate to 0.02500 @ T=2000!
Updating regularizer @ T=2000!
Epoch=0, BatchID=4800, Val_AUC=0.8726, Best_Val_AUC=0.8813
Epoch=0, BatchID=5200, Val_AUC=0.8811, Best_Val_AUC=0.8813
Epoch=0, BatchID=5600, Val_AU

In [16]:
root = '/content/'
testGen =  CheXpert(image_root_path=root, 
                    csv_path='valid_5.csv', 
                    image_size=224)

testloader =  torch.utils.data.DataLoader(testGen, batch_size=32, num_workers=2, shuffle=False)

In [17]:
model = WholeConv1dArch(model_1, model_2, conv1d)
model = model.cuda()
model.load_state_dict(torch.load(save_weights_path))

<All keys matched successfully>

In [18]:
model.eval()
with torch.no_grad():    
    test_pred = []
    test_true = [] 
    for data in testloader:
        test_data, test_labels = data
        test_data = test_data.cuda()
        y_pred = model(test_data)
        y_pred = torch.sigmoid(y_pred)
        test_pred.append(y_pred.cpu().detach().numpy())
        test_true.append(test_labels.numpy())
  
    test_true = np.concatenate(test_true)
    test_pred = np.concatenate(test_pred)

In [19]:
labels = ['Cardiomegaly', 'Edema', 'Consolidation', 'Atelectasis', 'Pleural Effusion']

results = pd.DataFrame(index=labels)


scores = []
for i in range(5):
    score = roc_auc_score(test_true[:, i], test_pred[:, i])
    scores.append(score)
    
results['AUC'] = scores

In [20]:
results

,AUC
Cardiomegaly,0.832331
Edema,0.923363
Consolidation,0.916728
Atelectasis,0.827822
Pleural Effusion,0.911119


In [21]:
results['AUC'].mean()

0.8822723755362217